# Ray Core - example workload

This default notebook is executed using Databricks Workflows as defined in databricks.yml.

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os, ray
head_ip = os.getenv("SPARK_LOCAL_IP")
print(f"Using Ray Head node at: {head_ip}")

ray.init(address=f'ray://{head_ip}:10001')

The below is arbitrary (useless) Ray code to demonstrate the concept.

In [0]:
@ray.remote
class Counter:
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value

# Create an actor from this class.
counter = Counter.remote()


obj_ref = counter.increment.remote()
print(ray.get(obj_ref))

# Create ten Counter actors.
counters = [Counter.remote() for _ in range(10)]

# Increment each Counter once and get the results. These tasks all happen in
# parallel.
results = ray.get([c.increment.remote() for c in counters])
print(results)

# Increment the first Counter five times. These tasks are executed serially
# and share state.
results = ray.get([counters[0].increment.remote() for _ in range(5)])
print(results)